Initialise arguments etc. Should output the folder containing the pretrained model, may be necessary to change data_path to an absolute path.

In [1]:
import opts
import argparse
import os
data_path = '../data_model/wikisql/'
model_path = os.path.join(data_path,'pretrain.pt')
parser = argparse.ArgumentParser(description='Exploration.ipynb')
opts.translate_opts(parser)
opt = parser.parse_args(
    ["-split","pred","-output","pred.txt","-data_path",data_path,"-model_path",model_path])

opt.anno = os.path.join(
    opt.data_path, 'annotated_ent/{}.jsonl'.format(opt.split))
opt.source_file = os.path.join(
    opt.data_path, 'data/{}.jsonl'.format(opt.split))
opt.db_file = os.path.join(opt.data_path, 'data/{}.db'.format(opt.split))
opt.pre_word_vecs = os.path.join(opt.data_path, 'embedding')
opt.data_path

'../data_model/wikisql/'

In [2]:
import torch
import table
import glob
import json
torch.cuda.set_device(opt.gpu)
dummy_parser = argparse.ArgumentParser(description='train.py')
opts.model_opts(dummy_parser)
opts.train_opts(dummy_parser)
dummy_opt = dummy_parser.parse_known_args([])[0]
fn_model = glob.glob(opt.model_path)[0]
opt.model=fn_model
fn_model

'../data_model/wikisql/pretrain.pt'

Load the pretrained model - may take time

In [3]:
checkpoint = torch.load(opt.model,
                                map_location=lambda storage, loc: storage)
fields = table.IO.TableDataset.load_fields(checkpoint['vocab'])

Start the translation model running. If the embedding folder is not set correctly this may attempt to download the GloVe vectors directly

In [4]:
translator = table.Translator(opt, dummy_opt.__dict__)

Loading model


Initialise spaCy,  requires the model installed as described in https://spacy.io/models/en#en_core_web_lg

In [5]:
import spacy
import codecs
import json
from spacy.tokens import Doc
nlp = spacy.load('en_core_web_lg')

Set up a new table. This example is from the training data, but can be modified. The rows are not used for the query, but at least one should exist.

In [6]:
tables={}
table_instance={"header": 
       ["Customer Id", "Name", "Address", "Priority", "Value", "Age", "Gender"],
       "page_title": "Customer applications", 
       "types": ["text", "text", "text", "text", "real", "real", "text"], 
       "id": "jca",
       "section_title": "Customer Details", 
       "caption": "Customer Details",
       "rows": [["1231234A", "UA", "2013-05-20", "2014-07-20", 180, 42, "Male"]], 
       "name": "table_jca"
      }
tables[table_instance['id']]=table_instance

Set up a question. The SQL field describes the expected query when training or testing. In this notebook it is not used, but it should still make sense for the table (e.g. conds should not specify a column number which is not in the table).

In [7]:
question = {"phase": 1, 
            "table_id": "jca", 
            "question": "How many customers have low age", 
            "sql": {"sel": 1, "conds": [[2, 0, "Williams"], [5, 0, "2"]], "agg": 3}
#            "sql": {"sel": 1, "conds": [], "agg": 1}
           }

This connects to a Stanford CoreNLP server to annotate the question. The server should be running 
on http://localhost:9000/ by default, this can be modified in annotate.py when instantiating CoreNLPClient.

In [8]:
from annotate import annotate_example
annotated_question = annotate_example(question, table_instance)
#annotated_question

Add POS annotations to question. 

In [9]:
w_list = annotated_question['question']['gloss']
ws_list = [it.isspace() for it in annotated_question['question']['after']]
doc = Doc(nlp.vocab, words=w_list, spaces=ws_list)
for name, proc in nlp.pipeline:
    doc = proc(doc)
annotated_question['question']['ent'] = [tk.tag_ for tk in doc]
assert(len(annotated_question['question']['ent']) == len(annotated_question['question']['words']))
#annotated_question['question']

In [10]:
annotated_question['question']

{'after': [' ', ' ', ' ', ' ', ' ', ''],
 'ent': ['WRB', 'JJ', 'NNS', 'VBP', 'JJ', 'NN'],
 'gloss': ['How', 'many', 'customers', 'have', 'low', 'age'],
 'words': ['how', 'many', 'customers', 'have', 'low', 'age']}

Actually generate the SQL.

In [11]:
js_list = [annotated_question]
data = table.IO.TableDataset(js_list, translator.fields, None, False)
test_data = table.IO.OrderedIterator(dataset=data,device=opt.gpu, batch_size=opt.batch_size, train=False, sort=True, sort_within_batch=False)
batch=next(iter(test_data))
result_list=translator.translate(batch)
pred = result_list[0]
pred.predict(annotated_question)

SELECT MIN col5 FROM table

In [35]:
result_list[0]

In [21]:
result_list[0].recover_cond_to_gloss(annotated_question)

[[3, 0, 'low priority']]

In [22]:
result_list[0].cond

[(3, 0, (3, 4))]

In [23]:
col, op, span = (result_list[0].cond)[0]
annotated_question['question']['gloss'],annotated_question['question']['after']

(['What', 'customers', 'have', 'low', 'priority'], [' ', ' ', ' ', ' ', ''])

In [27]:
pr = result_list[0]
pr.idx, pr.agg, pr.sel, pr.cond

(0, 0, 0, [(3, 0, (3, 4))])

Investigating incorrect select coulumn
